## General State Discrimination (based on [Chen et. al.](https://arxiv.org/abs/1805.08654))

Below we describe a scheme based on this [paper](https://arxiv.org/abs/1805.08654), used for discriminating between multiple family of states at accuracies comparable to the theoretically possible threshold when allowed error is minimized. The paper also claims when allowing slightly higher error, success accuracy is increased significantly at the cost of higher number of inconclusive signals. We notice a similar pattern in the examples below, although not as dramatic as claimed in the paper.

In [ ]:
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit import Aer
from qiskit.compiler import transpile

Below we create a general circuit using a total of 33 free parameters, allowing universal mapping from a 2 qubit state to two classifier qubits. The states 00,01 correspond to 1st family of staes, 10 to 2nd family of states and 11 is interpreted as an inconclusive signal.

In [25]:
from qiskit.circuit import ControlledGate
from qiskit.circuit.library import RYGate


## Main Classifer Circuit
def classifier_circuit():
    ## Initializing the circuit
    classifier_qubit = QuantumRegister(2)
    input_qubit = QuantumRegister(2)
    classifier_cbit = ClassicalRegister(2)
    qc = QuantumCircuit(classifier_qubit,input_qubit,classifier_cbit)
    
    ## All parametrized gates
    U1 = ParameterVector("U1",7)
    Ry1 = ParameterVector("Ry1",4)
    U2 = ParameterVector("U2",14)
    Ry2 = ParameterVector("Ry2",8)
    
    ## First gate: Arbitrary two qubit unitary
    qc.rz(U1[0],input_qubit[0])
    qc.rz(U1[1],input_qubit[1])
    qc.cx(input_qubit[1],input_qubit[0])
    qc.rz(U1[2],input_qubit[0])
    qc.ry(U1[3],input_qubit[1])
    qc.cx(input_qubit[0],input_qubit[1])
    qc.ry(U1[4],input_qubit[1])
    qc.cx(input_qubit[1],input_qubit[0])
    qc.rz(U1[5],input_qubit[0])
    qc.rz(U1[6],input_qubit[1])
    qc.barrier()
    
    
    ## Second gate: Two qubit controlled Ry rotation
    for i in range(4):
        bin_i = bin(i)[2:].zfill(2)

        rry = RYGate(Ry1[i])
        ccry = ControlledGate(name="mcry", num_qubits=3, params=rry.params, num_ctrl_qubits=2, definition=rry.definition,ctrl_state=bin_i, base_gate=rry) 

        qc.append(ccry,[input_qubit[0],input_qubit[1], classifier_qubit[0]])
        

    qc.barrier()
        
    ## Third gate: Controlled arbitrary two qubit unitary
    controls_U2 = [classifier_qubit[0]]

    for i in range(2):
        bin_i = bin(i)[2:].zfill(1)

        for ind,yes in enumerate(bin_i):
            if yes == '1':
                qc.x(controls_U2[ind])

            qc.crz(U2[7*i + 0],classifier_qubit[0],input_qubit[0])
            qc.crz(U2[7*i + 1],classifier_qubit[0],input_qubit[1])
            qc.ccx(input_qubit[1],classifier_qubit[0],input_qubit[0])
            qc.crz(U2[7*i + 2],classifier_qubit[0],input_qubit[0])
            qc.cry(U2[7*i + 3],classifier_qubit[0],input_qubit[1])
            qc.ccx(input_qubit[0],classifier_qubit[0],input_qubit[1])
            qc.cry(U2[7*i + 4],classifier_qubit[0],input_qubit[1])
            qc.ccx(input_qubit[1],classifier_qubit[0],input_qubit[0])
            qc.crz(U2[7*i + 5],classifier_qubit[0],input_qubit[0])
            qc.crz(U2[7*i + 6],classifier_qubit[0],input_qubit[1])

        for ind,yes in enumerate(bin_i):
            if yes == '1':
                qc.x(controls_U2[ind])

    qc.barrier()
        
    ## Fourth gate: Three qubit controlled Ry
    for i in range(8):
        bin_i = bin(i)[2:].zfill(3)

        rry = RYGate(Ry2[i])
        ccry = ControlledGate(name="mcry", num_qubits=4, params=rry.params, num_ctrl_qubits=3, definition=rry.definition,ctrl_state=bin_i, base_gate=rry) 

        qc.append(ccry,[classifier_qubit[0],input_qubit[0],input_qubit[1], classifier_qubit[1]])

    

    backend = Aer.get_backend('aer_simulator')
    qc1 = transpile(qc,backend = backend)
    
    return qc1, [U1,Ry1,U2,Ry2] 

Here we classify between these two family of states (one is pure, other is a mixed state):

1. $\psi_1 = a|00\rangle + \sqrt{1-a^2}|01\rangle$

2. $\rho_2 = \frac{1}{2}(|\psi_2^+\rangle\langle \psi_2^+| +|\psi_2^-\rangle\langle \psi_2^-|),\text{ where, }\psi_2 = b|01\rangle \pm \sqrt{1-b^2}|10\rangle$

Below function creates the states based on the label and corresponding parameter value

In [26]:
## Creates input data
def input_circ(label,var):
    qc = QuantumCircuit(2)
    theta = 2 * np.arccos(var)   
    
    if label == 0 or label == 1:
        qc.rx(theta,1)
        
    elif label == 2:
        sign = np.random.choice([1,-1])
        qc.rx(sign * theta,1)
        qc.cx(1,0)
        qc.x(0)
        
    return qc  

In [27]:
## This cell contains functions for running the circuit and measuring loss 
## functions for each iteration

def qnn(qc,parameter_obs,parameter_vals,shots = 1024):
    
    """Returns counts from circuit based on input state and classifier circuit"""
    parameters = []
    p1 = 0
    p2 = 0
    for ind in range(len(parameter_obs)):
        p1 = p2
        p2 = p2 + len(parameter_obs[ind])
        parameters.append(parameter_vals[p1:p2])
        
    all_params = {}
    for i in range(len(parameter_obs)):
        all_params[parameter_obs[i]] = parameters[i]
    
    qc1 = qc.bind_parameters(all_params)
    
    qregist = qc1.qregs[0][:2]
    cregist = qc1.cregs[0]

    qc1.measure(qregist,cregist)
    
    backend = Aer.get_backend("aer_simulator")
    counts = backend.run(qc1,shots = shots).result().get_counts()
    
    return counts
    
    
def compute_loss_each(counts,y,alp_err,alp_inc,shots):
    
    """Returns the value of loss function for each data point in training data"""
    P_suc = 0
    P_err = 0
    P_inc = 0
    
    for states in counts:
        if (int(states,2) == 0 or int(states,2) == 1) and (y==0 or y==1) :
            P_suc += counts[states]/shots
        
        elif int(states,2) == 2 and y == 2:
            P_suc += counts[states]/shots
            
        elif int(states,2) == 3:
            P_inc += counts[states]/shots
        
        else:
            P_err += counts[states]/shots
    
    return [(1 - P_suc) +  alp_err * P_err + alp_inc * P_inc, P_suc,P_err,P_inc]


def compute_loss(parameter_vals, X, y,classifier_circ,parameter_obs,alp_err,alp_inc,shots = 1024):
    
    """Returns average loss function over the entire training dataset.
    Also prints the associated values of training for every 30 iterations till convergence"""
    
    global P
    P += 1
  
    each_loss = 0
    each_suc = 0
    each_err = 0
    each_inc = 0
    
    
    for ind in range(len(y)):
        total_circ = QuantumCircuit(4,2)
        input_qc = input_circ(y[ind],X[ind])
        
        total_circ.append(input_qc,[2,3])
                
        total_circ.append(classifier_circ,range(4),range(2))
        backend = Aer.get_backend("aer_simulator")
        qc1 = transpile(total_circ, backend=backend)
        
        
        counts = qnn(qc1,parameter_obs,parameter_vals,shots)
        loss_info = compute_loss_each(counts,y[ind],alp_err,alp_inc,shots)
        each_loss += loss_info[0]
        each_suc += loss_info[1]
        each_err += loss_info[2]
        each_inc += loss_info[3]
        
    out = each_loss/len(y)
    
    succ = each_suc/len(y)
    err = each_err/len(y)
    inc = each_inc/len(y)
    
    if P%30 == 0:
        print(f"Iterations = {P}, Function value = {out : .4f}, Success = {succ: .4f}, Error = {err:.4f}, Inconc. = {inc: .4f}")
     
    return out

In [28]:
## Function for testing success rate on test data

def test_succ(X,y,parameter_vals):
    """Measures the success over test dataset.
    Returns test success and inconclusive signal rate."""
    classifier,parameter_obs = classifier_circuit()
    
    succ = 0
    inc = 0
    
    for i in range(len(y)):
        input_circ_test = input_circ(y[i],X[i])
        
        total_circ = QuantumCircuit(4,2)
        total_circ.append(input_circ_test,[2,3])
                
        total_circ.append(classifier,range(4),range(2))
        backend = Aer.get_backend("aer_simulator")
        qc1 = transpile(total_circ, backend=backend)
        
        
        counts = qnn(qc1,parameter_obs,parameter_vals,shots=1024)
        
        maxim = 0
        
        for states in counts:
            if counts[states] > maxim:
                maxim = counts[states]
                maxim_state = states
                
        dec_maxim = int(maxim_state,2)
        if y[i] == 0 or y[i] == 1:
            if dec_maxim == 0 or dec_maxim == 1:
                succ += 1
        
        elif y[i] == 2:
            if dec_maxim == 2:
                succ += 1
        
        if dec_maxim == 3:
            inc += 1
            
       
    return succ/len(y),inc/len(y)
        
        

## $a\in[0,1], b\in [0,1]$

In [ ]:
## Functions to create dataset based on the above distribution
def create_data(data_size):
    data_val = []
    data_label = []
    
    for _ in range(data_size):
        data_label.append(np.random.choice([0,1,2],p = [0.25,0.25,0.5]))
        data_val.append(np.random.uniform(0,1))
        
    return data_val, data_label

In [8]:
## Creating training and test dataset
X,y = create_data(100)
X_test,y_test = create_data(1000)

### We change the $\alpha_{err}, \alpha_{inc}$ values to understand the change in accuracy/error values

### $\alpha_{err} = 20, \alpha_{inc} = 10$

In [9]:
from scipy.optimize import minimize

alp_err,alp_inc = 20,10
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 33)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 30, Function value =  10.0597, Success =  0.4614, Error = 0.4135, Inconc. =  0.1252
Iterations = 60, Function value =  6.9974, Success =  0.5932, Error = 0.2522, Inconc. =  0.1546
Iterations = 90, Function value =  6.3756, Success =  0.6266, Error = 0.2268, Inconc. =  0.1466
Iterations = 120, Function value =  6.3862, Success =  0.6352, Error = 0.2373, Inconc. =  0.1275
Iterations = 150, Function value =  6.4484, Success =  0.6277, Error = 0.2353, Inconc. =  0.1371
Iterations = 180, Function value =  6.4901, Success =  0.6243, Error = 0.2357, Inconc. =  0.1400
Iterations = 210, Function value =  6.3674, Success =  0.6313, Error = 0.2312, Inconc. =  0.1375
Iterations = 240, Function value =  6.4612, Success =  0.6268, Error = 0.2356, Inconc. =  0.1375
Iterations = 270, Function value =  6.3455, Success =  0.6323, Error = 0.2301, Inconc. =  0.1376
Iterations = 300, Function value =  6.3911, Success =  0.6303, Error = 0.2325, Inconc. =  0.1372


In [10]:
test_succ(X_test,y_test,res_qnn.x)

(0.654, 0.0)

### $\alpha_{err} = 20, \alpha_{inc} = 15$

In [13]:
alp_err,alp_inc = 20,15
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 33)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 30, Function value =  4.8764, Success =  0.5049, Error = 0.3811, Inconc. =  0.1140
Iterations = 60, Function value =  4.0497, Success =  0.5745, Error = 0.2994, Inconc. =  0.1261
Iterations = 90, Function value =  3.9646, Success =  0.6075, Error = 0.3219, Inconc. =  0.0706
Iterations = 120, Function value =  3.6301, Success =  0.6388, Error = 0.2926, Inconc. =  0.0685
Iterations = 150, Function value =  3.7044, Success =  0.6316, Error = 0.2988, Inconc. =  0.0696
Iterations = 180, Function value =  3.5815, Success =  0.6424, Error = 0.2872, Inconc. =  0.0704
Iterations = 210, Function value =  3.7233, Success =  0.6296, Error = 0.3002, Inconc. =  0.0702
Iterations = 240, Function value =  3.8838, Success =  0.6149, Error = 0.3146, Inconc. =  0.0705
Iterations = 270, Function value =  3.6733, Success =  0.6337, Error = 0.2951, Inconc. =  0.0712
Iterations = 300, Function value =  4.0433, Success =  0.6003, Error = 0.3290, Inconc. =  0.0707


In [14]:
test_succ(X_test,y_test,res_qnn.x)

(0.654, 0.0)

Increasing $\alpha_{inc}$ keeps the accuracy same, although inconclusive rate on training data decreases as expected.

## $a\in [0,1], b\approx \frac{1}{\sqrt{2}} $

In [29]:
## Functions to create dataset based on the above distribution
def create_data(data_size):
    data_val = []
    data_label = []
    
    for _ in range(data_size):
        data_label.append(np.random.choice([0,1,2],p = [0.25,0.25,0.5]))
        if data_label[-1] == 0 or data_label[-1] == 1:
            data_val.append(np.random.uniform(0,1))
            
        elif data_label[-1] == 2:
            data_val.append(np.random.uniform(1/np.sqrt(2)-0.001,1/np.sqrt(2)+0.001))
    return data_val, data_label

In [30]:
## Creating training and test dataset
X,y = create_data(100)
X_test,y_test = create_data(1000)

### $\alpha_{err} = 20, \alpha_{inc} = 10$

In [33]:
from scipy.optimize import minimize

alp_err,alp_inc = 20,10
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 33)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 30, Function value =  8.0024, Success =  0.5200, Error = 0.2722, Inconc. =  0.2077
Iterations = 60, Function value =  4.8527, Success =  0.7371, Error = 0.1960, Inconc. =  0.0669
Iterations = 90, Function value =  5.2929, Success =  0.7133, Error = 0.2139, Inconc. =  0.0728
Iterations = 120, Function value =  5.2970, Success =  0.7147, Error = 0.2159, Inconc. =  0.0694
Iterations = 150, Function value =  5.1569, Success =  0.7223, Error = 0.2102, Inconc. =  0.0675
Iterations = 180, Function value =  5.0744, Success =  0.7253, Error = 0.2053, Inconc. =  0.0693
Iterations = 210, Function value =  5.1815, Success =  0.7212, Error = 0.2115, Inconc. =  0.0672
Iterations = 240, Function value =  5.3478, Success =  0.7129, Error = 0.2189, Inconc. =  0.0682
Iterations = 270, Function value =  5.4816, Success =  0.7056, Error = 0.2243, Inconc. =  0.0700


In [34]:
test_succ(X_test,y_test,res_qnn.x)

(0.751, 0.0)

### $\alpha_{err} = 20, \alpha_{inc} = 15$

In [31]:
from scipy.optimize import minimize

alp_err,alp_inc = 20,15
P = 0

init_guess = np.random.uniform(low = 0, high = 2*np.pi,size = 33)
classifier_circ,parameter_obs = classifier_circuit()
res_qnn = minimize(compute_loss,init_guess,method = 'COBYLA',args = (X, y,classifier_circ,parameter_obs,alp_err,alp_inc))

Iterations = 30, Function value =  7.4010, Success =  0.6044, Error = 0.2144, Inconc. =  0.1812
Iterations = 60, Function value =  5.8992, Success =  0.6987, Error = 0.2156, Inconc. =  0.0857
Iterations = 90, Function value =  6.6225, Success =  0.6741, Error = 0.2816, Inconc. =  0.0443
Iterations = 120, Function value =  6.3043, Success =  0.6880, Error = 0.2624, Inconc. =  0.0496
Iterations = 150, Function value =  5.3828, Success =  0.7333, Error = 0.2231, Inconc. =  0.0436
Iterations = 180, Function value =  6.0747, Success =  0.6986, Error = 0.2506, Inconc. =  0.0508
Iterations = 210, Function value =  5.5308, Success =  0.7259, Error = 0.2289, Inconc. =  0.0452
Iterations = 240, Function value =  5.4048, Success =  0.7324, Error = 0.2247, Inconc. =  0.0428
Iterations = 270, Function value =  5.4220, Success =  0.7313, Error = 0.2247, Inconc. =  0.0439


In [32]:
test_succ(X_test,y_test,res_qnn.x)

(0.742, 0.0)

Increasing $\alpha_{inc}$ decreases the accuracy and inconclusive rate on training data decreases as expected. The change however is not as dramatic as expected from the paper. This might be due to the optimzation being stuck in a local minima.